# TCC 1
* Aluno: Paulo Henrique Costa Gontijo
* Matrícula: 15/0143800


## Importação das Bibliotecas

In [2]:
import os

file_name = 'test.jsonl'
file_path = '/'.join([os.getcwd(), file_name])

## Criando Cluster Spark

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
        .builder \
        .appName("otto-reccomender-competition-kaggle") \
        .config("spark.sql.warehouse.dir", abspath('spark-warehouse')) \
        .config("spark.executor.cores", "10")\
        .config("spark.driver.cores", "10")\
        .config("spark.executor.instances", "2")\
        .config("spark.executor.memory","32g")\
        .config("spark.driver.memory", "32g")\
        .config("spark.default.parallelism","80")\
        .config("spark.sql.shuffle.partitions","20")\
        .config("spark.sql.autoBroadcastHashJoin","-1")\
        .config("spark.sql.files.maxPartitionBytes","1g")\
        .enableHiveSupport() \
        .getOrCreate()

# show configured parameters
print(SparkConf().getAll())

# set log level
spark.sparkContext.setLogLevel("FATAL")

23/10/10 14:35:17 WARN Utils: Your hostname, gobellek-B660M-DS3H-DDR4 resolves to a loopback address: 127.0.1.1; using 192.168.0.37 instead (on interface enp4s0)
23/10/10 14:35:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 14:35:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.sql.shuffle.partitions', '20'), ('spark.app.submitTime', '1696959317679'), ('spark.driver.memory', '32g'), ('spark.driver.cores', '10'), ('spark.default.parallelism', '80'), ('spark.executor.cores', '10'), ('spark.executor.memory', '32g'), ('spark.app.name', 'otto-reccomender-competition-kaggle'), ('spark.sql.autoBroadcastHashJoin', '-1'), ('spark.sql.catalogImplementation', 'hive'), ('spark.executor.instances', '2'), ('spark.sql.warehouse.dir', '/home/gobellek/Documents/UnB/Tcc/tcc/spark-warehouse'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.sql.files.maxPartitionBytes', '1g')]


* Conferindo o instanciamento

In [4]:
spark

* Processamento de Json para formato tabular

In [5]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [8]:
def explode_df_json_to_tabular(df_raw):
    df_explode = df_raw.withColumn('events_explode', F.explode('events'))\
                    .withColumn('session', F.col('session').cast(T.IntegerType()))\
                    .withColumn('aid', F.col('events_explode.aid').cast(T.IntegerType()))\
                    .withColumn('ts', F.col('events_explode.ts'))\
                    .withColumn('type', F.col('events_explode.type'))\
                    .drop('events', 'events_explode')
    return df_explode

* Categorização de coluna alvo string

In [9]:
from pyspark.ml.feature import StringIndexer

def transform_indexer(df_explode):
    indexer = StringIndexer(inputCol='type', outputCol='type_cat')
    indexer_fitted = indexer.fit(df_explode)
    df_indexed = indexer_fitted.transform(df_explode)
    df_indexed_type = df_indexed.withColumn('type_cat', F.col('type_cat').cast(T.IntegerType())).drop('type')
    df_indexed_type_renamed = df_indexed_type.withColumnRenamed('session','userCol')
    df_indexed_droped = df_indexed_type_renamed.dropDuplicates()
    df = df_indexed_droped.drop('ts')
    return df

# Carregando dataset de predição por categoria

In [21]:
def prepare_df_to_predictions(path):
    df_raw_carts = spark.read.format('json').load(path).persist()
    df_carts_exploded = df_raw_carts.withColumn('recommendations_explode', F.explode('recommendations'))\
                                    .withColumn('aid', F.col('recommendations_explode.aid').cast(T.IntegerType()))\
                                    .drop('recommendations_explode', 'recommendations')
    df_carts_submission = df_carts_exploded.withColumn('userCol', F.col('userCol').cast(T.StringType()))\
                                                .groupby('userCol').agg(F.collect_list('aid').alias("labels")).persist()
    return df_carts_submission.withColumnRenamed('userCol', 'session_type')

## Clicks

In [27]:
df_clicks = prepare_df_to_predictions('./raw-clicks-predictions.json').persist()
df_clicks.show(10, truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type|labels                                                                                                                                                                              |
+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12899781    |[754412, 1043508, 1336175, 759436, 950718, 121246, 1019736, 734026, 447, 1853288, 1469891, 901817, 1833745, 1767530, 199008, 1008624, 1009427, 83856, 100012, 3542]                 |
|12899783    |[1849490, 1849480, 1849456, 1849450, 1849446, 1849436, 1849426, 1849406, 1849400, 1849380, 1849366, 1849346, 1849326, 1849310, 1849296, 1849276, 1849270, 1849250, 1849240, 1849230]|
|12899793    |[40566

In [76]:
df_clicks.count()

1670191

## Carts

In [26]:
df_carts = prepare_df_to_predictions('./raw-carts-predictions.json').persist()
df_carts.show(10, truncate=False)

+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type|labels                                                                                                                                                                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12899781    |[754412, 1043508, 759436, 950718, 1336175, 1019736, 447, 734026, 1853288, 121246, 1469891, 1833745, 1767530, 199008, 1008624, 3542, 901817, 1009427, 474743, 634452]         |
|12899887    |[1025795, 835431, 1620020, 1270528, 1201242, 1019116, 892871, 698925, 1804260, 1030727, 501077, 1676344, 1617771, 810494, 1763383, 1176975, 879201, 480861, 398437, 1458715] |
|12900014    |[1571865, 52715, 482711, 395217, 1215467,

In [77]:
df_carts.count()

242833

## Orders

In [29]:
df_orders = prepare_df_to_predictions('./raw-orders-predictions.json').persist()
df_orders.show(10, truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type|labels                                                                                                                                                                        |
+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12899793    |[846545, 1495817, 688858, 1132001, 1051747, 976467, 432989, 471073, 479903, 399315, 1678293, 988819, 623226, 984146, 180910, 710786, 832192, 1442599, 1691301, 920378]        |
|12900138    |[1580498, 1382441, 75686, 1588163, 714524, 1219503, 1634198, 807569, 559447, 1602299, 1078290, 258353, 1533875, 155079, 1744684, 1836735, 529056, 289665, 1193887, 834]       |
|12900216    |[1236775, 145332, 332654, 1699089, 1

In [30]:
df_orders.count()

35849

# Leitura de Dataset RAW_PREDICTIONS

In [34]:
df_general_recommend = prepare_df_to_predictions('./raw_predictions.json').persist()
df_general_recommend.show(10, truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type|labels                                                                                                                                                                              |
+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12899781    |[754412, 1043508, 759436, 447, 950718, 1019736, 121246, 923948, 734026, 1767530, 1833745, 199008, 1853288, 1008624, 493104, 1723428, 536184, 1336175, 3542, 634452]                 |
|12899783    |[1848840, 1848836, 1848820, 1848816, 1848810, 1848796, 1848776, 1848770, 1848750, 1848736, 1848720, 1848710, 1848706, 1848676, 1848660, 1848656, 1848650, 1848640, 1848626, 1848616]|
|12899793    |[40566

# Criando datasets complementares
## Clicks

In [74]:
df_general_clicks = df_general_recommend.join(df_clicks, on='session_type', how='left_anti')
df_general_clicks.count()

1612

## Carts

In [75]:
df_general_carts = df_general_recommend.join(df_carts, on='session_type', how='left_anti')
df_general_carts.count()

1428970

## Orders

In [78]:
df_general_orders = df_general_recommend.join(df_orders, on='session_type', how='left_anti')
df_general_orders.count()

1635954

# Agrupando dados de usuários e produtos
Pré-processamento do dataset de submissão.

## Clicks

In [122]:
users_clicks = [r[0] for r in df_clicks.select('session_type').distinct().toLocalIterator()]
users_general_clicks = [r[0] for r in df_general_clicks.select('session_type').distinct().toLocalIterator()]

In [123]:
users_list_clicks = [s + '_clicks' for s in users_clicks]
users_list_general_clicks = [s + '_clicks' for s in users_general_clicks]

In [124]:
labels_clicks = [r[0] for r in df_clicks.select('labels').toLocalIterator()]
labels_general_clicks = [r[0] for r in df_general_clicks.select('labels').toLocalIterator()]

In [125]:
df_clicks_submission = spark.createDataFrame(zip(users_list_clicks, labels_clicks), ['session_type', 'labels'])
df_clicks_submission.count()

1670191

In [126]:
df_clicks_general_submission = spark.createDataFrame(zip(users_list_general_clicks, labels_general_clicks), ['session_type', 'labels'])
df_clicks_general_submission.count()

1612

## Carts

In [127]:
users_carts = [r[0] for r in df_carts.select('session_type').distinct().toLocalIterator()]
users_general_carts = [r[0] for r in df_general_carts.select('session_type').distinct().toLocalIterator()]

In [128]:
users_list_carts = [s + '_carts' for s in users_carts]
users_list_general_carts = [s + '_carts' for s in users_general_carts]

In [129]:
labels_carts = [r[0] for r in df_carts.select('labels').toLocalIterator()]
labels_general_carts = [r[0] for r in df_general_carts.select('labels').toLocalIterator()]

In [130]:
df_carts_submission = spark.createDataFrame(zip(users_list_carts, labels_carts), ['session_type', 'labels'])
df_carts_submission.count()

242833

In [131]:
df_carts_general_submission = spark.createDataFrame(zip(users_list_general_carts, labels_general_carts), ['session_type', 'labels'])
df_carts_general_submission.count()

1428970

## Orders

In [132]:
users_orders = [r[0] for r in df_orders.select('session_type').distinct().toLocalIterator()]
users_general_orders = [r[0] for r in df_general_orders.select('session_type').distinct().toLocalIterator()]

In [133]:
users_list_orders = [s + '_orders' for s in users_orders]
users_list_general_orders = [s + '_orders' for s in users_general_orders]

In [134]:
labels_orders = [r[0] for r in df_orders.select('labels').toLocalIterator()]
labels_general_orders = [r[0] for r in df_general_orders.select('labels').toLocalIterator()]

In [135]:
df_orders_submission = spark.createDataFrame(zip(users_list_orders, labels_orders), ['session_type', 'labels'])
df_orders_submission.count()

35849

In [136]:
df_orders_general_submission = spark.createDataFrame(zip(users_list_general_orders, labels_general_orders), ['session_type', 'labels'])
df_orders_general_submission.count()

1635954

## Checando amostragem

In [137]:
df_clicks_submission.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12899781_clicks|[754412, 1043508,...|
|12899783_clicks|[1849490, 1849480...|
|12899793_clicks|[405663, 846545, ...|
|12899830_clicks|[1849490, 1849480...|
|12899872_clicks|[1849490, 1849480...|
+---------------+--------------------+
only showing top 5 rows



In [138]:
df_clicks_general_submission.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12972185_clicks|[191614, 165160, ...|
|12980101_clicks|[714968, 544144, ...|
|13012137_clicks|[226009, 1476166,...|
|13014853_clicks|[714524, 836852, ...|
|13061838_clicks|[352192, 1156699,...|
+---------------+--------------------+
only showing top 5 rows



In [139]:
df_carts_submission.show(5)

+--------------+--------------------+
|  session_type|              labels|
+--------------+--------------------+
|12899781_carts|[754412, 1043508,...|
|12899887_carts|[1025795, 835431,...|
|12900014_carts|[1571865, 52715, ...|
|12900080_carts|[1495817, 29735, ...|
|12900099_carts|[986164, 298371, ...|
+--------------+--------------------+
only showing top 5 rows



In [140]:
df_carts_general_submission.show(5)

+--------------+--------------------+
|  session_type|              labels|
+--------------+--------------------+
|12899783_carts|[1848840, 1848836...|
|12899793_carts|[405663, 846545, ...|
|12899830_carts|[1848840, 1848836...|
|12899872_carts|[1848840, 1848836...|
|12899968_carts|[1848840, 1848836...|
+--------------+--------------------+
only showing top 5 rows



In [141]:
df_orders_submission.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12899793_orders|[846545, 1495817,...|
|12900138_orders|[1580498, 1382441...|
|12900216_orders|[1236775, 145332,...|
|12900288_orders|[986294, 1699089,...|
|12900431_orders|[122983, 1512753,...|
+---------------+--------------------+
only showing top 5 rows



In [142]:
df_orders_general_submission.show(5)

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12899781_orders|[754412, 1043508,...|
|12899783_orders|[1848840, 1848836...|
|12899830_orders|[1848840, 1848836...|
|12899872_orders|[1848840, 1848836...|
|12899887_orders|[1025795, 1620020...|
+---------------+--------------------+
only showing top 5 rows



# Concatenando Dataset

In [143]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [df_clicks_submission,
       df_clicks_general_submission,
       df_carts_submission,
       df_carts_general_submission,
       df_orders_submission,
       df_orders_general_submission]

df_submission = reduce(DataFrame.unionAll, dfs).persist()
df_submission = df_submission.withColumn("labels",F.concat_ws(" ",F.col("labels")))
df_submission.show()

+---------------+--------------------+
|   session_type|              labels|
+---------------+--------------------+
|12899781_clicks|754412 1043508 13...|
|12899783_clicks|1849490 1849480 1...|
|12899793_clicks|405663 846545 116...|
|12899830_clicks|1849490 1849480 1...|
|12899872_clicks|1849490 1849480 1...|
|12899887_clicks|1025795 835431 16...|
|12899968_clicks|1849490 1849480 1...|
|12899996_clicks|1849490 1849480 1...|
|12900014_clicks|1571865 482711 52...|
|12900049_clicks|1849490 1849480 1...|
|12900055_clicks|1849490 1849480 1...|
|12900080_clicks|1116095 1270528 1...|
|12900099_clicks|986164 298371 274...|
|12900116_clicks|1849490 1849480 1...|
|12900121_clicks|319580 868327 179...|
|12900122_clicks|1849490 1849480 1...|
|12900130_clicks|1849490 1849480 1...|
|12900137_clicks|1849490 1849480 1...|
|12900138_clicks|1580498 1219503 7...|
|12900202_clicks|1849490 1849480 1...|
+---------------+--------------------+
only showing top 20 rows



In [144]:
df_submission.count()

5015409

In [146]:
df_submission.orderBy('session_type').show(truncate=False)

+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|session_type   |labels                                                                                                                                                         |
+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12899779_carts |1848840 1848836 1848820 1848816 1848810 1848796 1848776 1848770 1848750 1848736 1848720 1848710 1848706 1848676 1848660 1848656 1848650 1848640 1848626 1848616|
|12899779_clicks|1849490 1849480 1849456 1849450 1849446 1849436 1849426 1849406 1849400 1849380 1849366 1849346 1849326 1849310 1849296 1849276 1849270 1849250 1849240 1849230|
|12899779_orders|1848840 1848836 1848820 1848816 1848810 1848796 1848776 1848770 1848750 1848736 1848720 18487

# Escrevendo dataset de submissão

In [147]:
df_submission.orderBy('session_type').coalesce(1)\
                                     .write.format('csv')\
                                     .mode("overwrite")\
                                     .option("header", True)\
                                     .save('/'.join([os.getcwd(), 'submission']))

# Referências Bibliográficas

> https://github.com/Kaggle/kaggle-api, acessado em 24/01/2023.

> https://www.youtube.com/watch?v=aBNQzWV_UmE, acessado em 24/01/2023.

> https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/968100988546031/157591980591166/8836542754149149/latest.html, acessado em 25/01/2023.

> https://blog.clairvoyantsoft.com/apache-spark-out-of-memory-issue-b63c7987fff, acessado em 25/01/2023.

> https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html, acessado em 26/01/2023.

> https://docs.fast.ai/collab.html, acessado em 24/01/2023.

> https://course.fast.ai/Lessons/lesson7.html, acessado em 24/01/2023.

> https://github.com/fastai/fastbook/blob/master/08_collab.ipynb, acessado em 24/01/2023.

> https://www.kaggle.com/code/vchulski/tutorial-collaborative-filtering-with-pyspark

> https://medium.com/analytics-vidhya/crafting-recommendation-engine-in-pyspark-a7ca242ad40a

> https://www.freecodecamp.org/news/8-clustering-algorithms-in-machine-learning-that-all-data-scientists-should-know/

> https://www.youtube.com/watch?v=fpqa0_U4zb4

> https://www.youtube.com/watch?v=58OjaDH2FI0&t=742s

> https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

> https://spark.apache.org/docs/latest/configuration.html#memory-management